# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [60]:
import pandas as pd
import math

import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import preprocessing
from sklearn import utils
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from stepwise_regression import step_reg
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mlxtend.feature_selection import SequentialFeatureSelector

from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols

import patsy

import scipy.stats

In [61]:
df = pd.read_csv('previsao_de_renda.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [65]:
df = pd.read_csv('previsao_de_renda.csv')
df.drop(['Unnamed: 0', 'id_cliente', 'data_ref'], axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [66]:
X = pd.get_dummies(df.drop('renda', axis=1), drop_first = True)
y = df['renda']
X.values.reshape(1,-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, 
                                                    train_size= 0.75, random_state = 42)

2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

In [67]:
ridge = Ridge()

parameters = {'alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1]}
ridge_regression = GridSearchCV(ridge, parameters, scoring='r2', cv=None)
ridge_regression.fit(X_test, y_test)
print(ridge_regression.best_params_)
print(ridge_regression.best_score_)

{'alpha': 0.1}
0.2847067205249384


3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?


In [68]:
lasso = Lasso()

parameters = {'alpha':[0, 0.001, 0.005, 0.01, 0.05, 0.1]}
lasso_regression = GridSearchCV(lasso, parameters, scoring='r2', cv=None)
lasso_regression.fit(X_test, y_test)


C:\Users\feldb\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\feldb\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\feldb\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.460e+10, tolerance: 1.295e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = 

C:\Users\feldb\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.013e+10, tolerance: 1.476e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\feldb\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.953e+10, tolerance: 1.438e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\feldb\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.081e+10, toleranc

GridSearchCV(estimator=Lasso(),
             param_grid={'alpha': [0, 0.001, 0.005, 0.01, 0.05, 0.1]},
             scoring='r2')

In [69]:
print(lasso_regression.best_params_)
print(lasso_regression.best_score_)

{'alpha': 0.1}
0.2846697676746014


4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?

In [70]:
df = pd.read_csv('previsao_de_renda.csv')
df.drop(['Unnamed: 0', 'id_cliente', 'data_ref'], axis=1, inplace=True)
df.dropna(inplace=True)
X = pd.get_dummies(df1.drop('renda', axis=1), drop_first = True)
y = df['renda']
X = X.astype(int)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, 
                                                    train_size= 0.75, random_state = 42)

In [72]:
X_test.shape

(3107, 24)

In [75]:
y_test = list(y_test)
y_test

[1705.55,
 1748.99,
 1733.67,
 2378.25,
 1957.87,
 1218.39,
 2085.39,
 8849.5,
 4676.05,
 1915.48,
 3356.47,
 4264.23,
 2247.23,
 1666.38,
 5250.55,
 2872.99,
 5025.98,
 8522.41,
 2416.77,
 7620.51,
 1820.84,
 2684.77,
 1043.34,
 21332.93,
 5045.4,
 3906.16,
 1728.92,
 17555.6,
 3405.61,
 8929.22,
 2928.12,
 3421.45,
 4045.44,
 18805.6,
 4742.33,
 1486.95,
 2081.38,
 14385.5,
 3707.71,
 4154.05,
 1240.38,
 3604.4,
 18298.97,
 2974.18,
 27089.27,
 5240.19,
 2037.0,
 2923.9,
 17376.58,
 6166.65,
 1547.03,
 3349.37,
 1174.45,
 6579.83,
 4221.19,
 564.78,
 5834.37,
 15375.48,
 3037.0,
 1878.91,
 1458.83,
 3362.06,
 15919.06,
 1303.38,
 2802.44,
 2440.24,
 3255.03,
 9530.44,
 1407.43,
 1829.94,
 4855.44,
 3505.43,
 2317.6,
 23013.28,
 6772.53,
 9001.2,
 1800.49,
 462.43,
 371.94,
 2725.28,
 13425.06,
 11032.31,
 8048.85,
 1052.59,
 3011.85,
 5302.23,
 6806.95,
 6100.4,
 2379.06,
 3398.81,
 1899.62,
 3302.63,
 2644.74,
 2139.6,
 4276.78,
 1090.68,
 9254.05,
 3971.34,
 4788.9,
 1122.29,
 9795

In [76]:
model = sm.OLS(y_test, X_test)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.572
Model:                            OLS   Adj. R-squared (uncentered):              0.569
Method:                 Least Squares   F-statistic:                              179.1
Date:                Mon, 06 Mar 2023   Prob (F-statistic):                        0.00
Time:                        18:37:49   Log-Likelihood:                         -31576.
No. Observations:                3107   AIC:                                  6.320e+04
Df Residuals:                    3084   BIC:                                  6.334e+04
Df Model:                          23                                                  
Covariance Type:            nonrobust                                                  
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
posse_de_veiculo                620.3598    251.164      2.470      0.014     127.894    1112.825
posse_de_imovel                 373.2743    243.141      1.535      0.125    -103.460     850.009
qtd_filhos                     1826.0057    810.162      2.254      0.024     237.495    3414.517
idade                            50.0341     14.035      3.565      0.000      22.516      77.552
tempo_emprego                   500.5046     18.460     27.113      0.000     464.310     536.699
qt_pessoas_residencia         -1599.5924    783.873     -2.041      0.041   -3136.559     -62.626
sexo_M                         5551.7584    257.955     21.522      0.000    5045.978    6057.538
tipo_renda_Bolsista           -1.303e-13   9.25e-13     -0.141      0.888   -1.94e-12    1.68e-12
tipo_renda_Empresário           670.3056    257.436      2.604      0.009     165.543    1175.069
tipo_renda_Pensionista        -2532.4317   4464.857     -0.567      0.571   -1.13e+04    6221.963
tipo_renda_Servidor público     243.6870    393.077      0.620      0.535    -527.032    1014.406
educacao_Pós graduação         1904.3485   2350.586      0.810      0.418   -2704.524    6513.221
educacao_Secundário             360.9991   1242.924      0.290      0.771   -2076.044    2798.043
educacao_Superior completo     1185.2046   1248.505      0.949      0.343   -1262.782    3633.191
educacao_Superior incompleto     69.7507   1343.261      0.052      0.959   -2564.025    2703.527
estado_civil_Separado         -1670.8368    912.701     -1.831      0.067   -3460.401     118.727
estado_civil_Solteiro         -2170.4910    820.376     -2.646      0.008   -3779.030    -561.952
estado_civil_União             -284.9811    454.937     -0.626      0.531   -1176.991     607.028
estado_civil_Viúvo            -1945.8824   1038.404     -1.874      0.061   -3981.916      90.152
tipo_residencia_Casa            145.7936    881.245      0.165      0.869   -1582.094    1873.681
tipo_residencia_Com os pais    -191.3341    992.820     -0.193      0.847   -2137.989    1755.321
tipo_residencia_Comunitário     -20.4382   1582.373     -0.013      0.990   -3123.051    3082.174
tipo_residencia_Estúdio        3406.4320   1803.279      1.889      0.059    -129.317    6942.181
tipo_residencia_Governamental   842.0091   1099.517      0.766      0.444   -1313.851    2997.870
==============================================================================
Omnibus:                     2748.909   Durbin-Watson:                   2.060
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           141667.736
Skew:                           3.996   Prob(JB):                         0.00
Kurtosis:             

In [77]:
backselect = step_reg.backward_regression(X_test, y_test, 0.05,verbose=False)
backselect

['posse_de_veiculo',
 'qtd_filhos',
 'idade',
 'tempo_emprego',
 'sexo_M',
 'tipo_renda_Empresário',
 'educacao_Superior completo',
 'tipo_residencia_Estúdio']

In [80]:
y = list(y)

In [81]:
X_backselect = X[['posse_de_veiculo',
 'qtd_filhos',
 'idade',
 'tempo_emprego',
 'sexo_M',
 'tipo_renda_Empresário',
 'educacao_Superior completo',
 'tipo_residencia_Estúdio']]

backmodel = sm.OLS(y, X_backselect)
backres = backmodel.fit()
backres.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.491
Model:                            OLS   Adj. R-squared (uncentered):              0.491
Method:                 Least Squares   F-statistic:                              1500.
Date:                Mon, 06 Mar 2023   Prob (F-statistic):                        0.00
Time:                        18:38:25   Log-Likelihood:                     -1.2883e+05
No. Observations:               12427   AIC:                                  2.577e+05
Df Residuals:                   12419   BIC:                                  2.577e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
posse_de_veiculo             -35.2248    148.010     -0.238      0.812    -325.347     254.898
qtd_filhos                   -48.2333     85.554     -0.564      0.573    -215.933     119.467
idade                         -4.9706      3.419     -1.454      0.146     -11.673       1.732
tempo_emprego                553.5392     10.951     50.548      0.000     532.074     575.005
sexo_M                      5817.6929    151.526     38.394      0.000    5520.679    6114.707
tipo_renda_Empresário        598.9269    152.777      3.920      0.000     299.461     898.393
educacao_Superior completo   427.2590    140.878      3.033      0.002     151.117     703.401
tipo_residencia_Estúdio      480.2440    893.352      0.538      0.591   -1270.864    2231.352
==============================================================================
Omnibus:                    17669.318   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10048577.290
Skew:                           8.217   Prob(JB):                         0.00
Kurtosis:                     141.335   Cond. No.                         549.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [82]:
forwardselect = step_reg.forward_regression(X_test, y_test, 0.05,verbose=False)
forwardselect

C:\Users\feldb\anaconda3\lib\site-packages\stepwise_regression\step_reg.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
C:\Users\feldb\anaconda3\lib\site-packages\stepwise_regression\step_reg.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
C:\Users\feldb\anaconda3\lib\site-packages\stepwise_regression\step_reg.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
C:\Users\feldb\anaconda3\lib\site-packages\stepwise_regression\step_reg.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float

['tempo_emprego',
 'sexo_M',
 'educacao_Superior completo',
 'idade',
 'posse_de_veiculo',
 'tipo_renda_Empresário',
 'qt_pessoas_residencia',
 'tipo_residencia_Estúdio']

In [83]:
X_forwardselect = X[['tempo_emprego',
 'sexo_M',
 'educacao_Superior completo',
 'idade',
 'posse_de_veiculo',
 'tipo_renda_Empresário',
 'qt_pessoas_residencia',
 'tipo_residencia_Estúdio']]

forwardmodel = sm.OLS(y, X_forwardselect)
forwards = forwardmodel.fit()
forwards.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.492
Model:                            OLS   Adj. R-squared (uncentered):              0.491
Method:                 Least Squares   F-statistic:                              1501.
Date:                Mon, 06 Mar 2023   Prob (F-statistic):                        0.00
Time:                        18:38:41   Log-Likelihood:                     -1.2883e+05
No. Observations:               12427   AIC:                                  2.577e+05
Df Residuals:                   12419   BIC:                                  2.577e+05
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
tempo_emprego                553.4760     10.948     50.556      0.000     532.017     574.935
sexo_M                      5862.1350    153.058     38.300      0.000    5562.117    6162.153
educacao_Superior completo   456.8915    141.489      3.229      0.001     179.551     734.232
idade                         -0.1020      4.265     -0.024      0.981      -8.463       8.259
posse_de_veiculo               5.4593    149.285      0.037      0.971    -287.163     298.081
tipo_renda_Empresário        612.9827    152.928      4.008      0.000     313.221     912.744
qt_pessoas_residencia       -123.3558     62.259     -1.981      0.048    -245.393      -1.319
tipo_residencia_Estúdio      471.4677    893.221      0.528      0.598   -1279.384    2222.320
==============================================================================
Omnibus:                    17652.657   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10010782.201
Skew:                           8.202   Prob(JB):                         0.00
Kurtosis:                     141.074   Cond. No.                         550.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?


Considerando o R², o melhor modelo foi o de regressão linear com todas as variáveis, pois, apresentou o melhor R².

Ridge: 
    
R² 0.2847067205249384

Lasso:
R² 0.2846697676746014

Regressão Linear:
    
R² 0.572

Backward
R² 0.491

Forward: 

R² 0.492


6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.


R: Removendo as variáveis comp-value superior a 0.05 não houve alteração no R² dos modelos.

In [121]:
X_backselect = X[[
 'tempo_emprego',
 'sexo_M',
 'tipo_renda_Empresário',
 'educacao_Superior completo']]

backmodel = sm.OLS(y, X_backselect)
backres = backmodel.fit()
backres.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.491
Model:                            OLS   Adj. R-squared (uncentered):              0.491
Method:                 Least Squares   F-statistic:                              3000.
Date:                Mon, 06 Mar 2023   Prob (F-statistic):                        0.00
Time:                        19:55:31   Log-Likelihood:                     -1.2884e+05
No. Observations:               12427   AIC:                                  2.577e+05
Df Residuals:                   12423   BIC:                                  2.577e+05
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
tempo_emprego                541.4504      8.173     66.253      0.000     525.431     557.470
sexo_M                      5703.0445    127.757     44.640      0.000    5452.620    5953.469
tipo_renda_Empresário        515.5418    144.446      3.569      0.000     232.406     798.678
educacao_Superior completo   343.8478    132.325      2.599      0.009      84.470     603.226
==============================================================================
Omnibus:                    17732.576   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10178345.448
Skew:                           8.274   Prob(JB):                         0.00
Kurtosis:                     142.224   Cond. No.                         22.2
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [122]:
X_forwardselect = X[['tempo_emprego',
 'sexo_M',
 'educacao_Superior completo',
 'tipo_renda_Empresário',
 'qt_pessoas_residencia']]

forwardmodel = sm.OLS(y, X_forwardselect)
forwards = forwardmodel.fit()
forwards.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.492
Model:                            OLS   Adj. R-squared (uncentered):              0.491
Method:                 Least Squares   F-statistic:                              2402.
Date:                Mon, 06 Mar 2023   Prob (F-statistic):                        0.00
Time:                        19:56:53   Log-Likelihood:                     -1.2883e+05
No. Observations:               12427   AIC:                                  2.577e+05
Df Residuals:                   12422   BIC:                                  2.577e+05
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
tempo_emprego                553.4660      9.378     59.018      0.000     535.084     571.848
sexo_M                      5867.4336    142.407     41.202      0.000    5588.294    6146.574
educacao_Superior completo   458.3624    139.378      3.289      0.001     185.161     731.564
tipo_renda_Empresário        613.7503    149.232      4.113      0.000     321.233     906.267
qt_pessoas_residencia       -123.8444     47.441     -2.611      0.009    -216.835     -30.853
==============================================================================
Omnibus:                    17651.579   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         10006954.744
Skew:                           8.201   Prob(JB):                         0.00
Kurtosis:                     141.048   Cond. No.                         22.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [110]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

renda = pd.read_csv('previsao_de_renda.csv')
renda.drop(['Unnamed: 0', 'id_cliente'], axis=1)
renda['posse_de_veiculo'] = renda['posse_de_veiculo'].map({True: 1,False: 0})
renda['posse_de_imovel'] = renda['posse_de_imovel'].map({True: 1,False: 0})
renda['data_ref'] = pd.to_datetime(renda['data_ref'])
renda['data_ref'] = renda['data_ref'].dt.strftime('%m-%Y')
            
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, 
                                                    train_size= 0.75, random_state = 42)

regr = DecisionTreeRegressor(max_depth=2)

regr.fit(X_train, y_train)

path = regr.cost_complexity_pruning_path(X_train, y_train)
path

ccp_alphas, impurities = path.ccp_alphas, path.impurities

clfs = []

for ccp_alpha in ccp_alphas:
    
    clf = DecisionTreeRegressor(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
    
arvores = []

for ccp_alpha in ccp_alphas:
    clf = DecisionTreeRegressor(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    arvores.append(clf)

arvores

train_scores = [mean_squared_error(y_train , clf.predict(X_train)) for clf in arvores]
test_scores  = [mean_squared_error(y_test  , clf.predict(X_test )) for clf in arvores]

train_scores

[15792549.065815398, 51041512.2945975, 66097321.548941225]

In [111]:
test_scores

[49956117.11155751, 41839049.03019456, 46317122.993746795]

In [119]:
arvore_final = DecisionTreeRegressor(random_state = 0)
arvore_final.fit(X_train, y_train)
print(f"R-quadrado na base de treino: {arvore_final.score(X_train, y_train):.2f}")

R-quadrado na base de treino: 0.82


In [120]:
arvore_final = DecisionTreeRegressor(random_state = 0)
arvore_final.fit(X_test, y_test)
print(f"R-quadrado na base de testes: {arvore_final.score(X_test, y_test):.2f}")

R-quadrado na base de testes: 0.88
